# Migrating tf.summary usage to TF 2.x

> Note: 本文档适用于已经熟悉TensorFlow 1.x TensorBoard并希望将大型TensorFlow代码库从TensorFlow 1.x迁移到2.x的人。如果您是TensorBoard的新手，请参阅入门文档。如果您正在使用tf.keras，您可能不需要采取任何行动来升级到TensorFlow 2.x。

In [1]:
import tensorflow as tf

TensorFlow 2.x包括对 `tf.summary` API的重大更改，该API用于在 TensorBoard 中编写摘要数据进行可视化。

# What's changed

将 `tf.summary` API视为两个子API是有用的：

- 一套用于记录 individual summaries 的操作 - `summary.scalar()`, `summary.histogram()`, `summary.image()`, `summary.audio()` 和 `summary.text()`。
- 编写逻辑，收集这些单独的摘要并将其写入特殊格式的日志文件（然后TensorBoard读取该文件以生成可视化）。

## In TF1.x

这两部分必须手动连接在一起-通过 `Session.run()` 获取摘要操作输出并调用 `FileWriter.add_summary(output，step)`。 `v1.summary.merge_all()` op 通过使用 graph 集合来聚合所有摘要操作输出，但这种方法对于 eager 执行和控制流仍然不起作用，使其特别不适合TF 2.x。

## In TF2.X

这两部分紧密集成，现在单个 `tf.summary` 操作在执行时会立即写入数据。使用模型代码中的API应该看起来仍然很熟悉，但现在在保持 graph-mode 兼容的同时，eager 执行是友好的。集成两个API意味着 `summary.FileWriter` 现在是 TensorFlow 执行上下文的一部分，可以直接由 `tf.summary` 操作访问，因此 configuring writers 是看起来不同的主要部分。

eager 执行的用法示例，TF 2.x中的默认值：

In [2]:
writer = tf.summary.create_file_writer("/tmp/mylogs/eager")

with writer.as_default():
    for step in range(100):
        tf.summary.scalar("my_metric", 0.5, step=step)
        writer.flush()

Metal device set to: Apple M1


2022-04-11 11:03:07.722744: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-11 11:03:07.723758: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
ls /tmp/mylogs/eager

events.out.tfevents.1649646187.Shawns.local.37805.0.v2


`tf.function` graph 执行的示例用法：

In [4]:
writer = tf.summary.create_file_writer("/tmp/mylogs/tf_function")

@tf.function
def my_func(step):
    with writer.as_default():
        tf.summary.scalar("my_metric", 0.5, step=step)
        
for step in tf.range(100, dtype=tf.int64):
    my_func(step)
    writer.flush()

2022-04-11 11:05:31.956152: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-11 11:05:31.958669: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-11 11:05:31.959817: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [5]:
ls /tmp/mylogs/tf_function

events.out.tfevents.1649646331.Shawns.local.37805.1.v2


传统 TF 1.x graph 执行的示例：

In [8]:
g = tf.compat.v1.Graph()
with g.as_default():
    step = tf.Variable(0, dtype=tf.int64)
    step_update = step.assign_add(1)
    writer = tf.summary.create_file_writer("/tmp/mylogs/session")
    with writer.as_default():
        tf.summary.scalar("my_metric", 0.5, step=step)
    all_summary_ops = tf.compat.v1.summary.all_v2_summary_ops()
    writer_flush = writer.flush()
    
with tf.compat.v1.Session(graph=g) as sess:
    sess.run([writer.init(), step.initializer])
    
    for i in range(100):
        sess.run(all_summary_ops)
        sess.run(step_update)
        sess.run(writer_flush)

2022-04-11 11:09:41.660502: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-11 11:09:41.660527: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-11 11:09:41.662150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-11 11:09:41.667329: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-11 11:09:41.670129: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-11 11:09:41.67

In [9]:
ls /tmp/mylogs/session

events.out.tfevents.1649646581.Shawns.local.37805.2.v2


# Converting your code

将现有的 `tf.summary` 使用转换为TF 2.x API无法可靠地自动化，因此 `tf_upgrade_v2` 脚本只是将其全部重写为 `tf.compat.v1.summary`，并且不会自动启用 TF 2.x行为。

## Partial  Migration

为了使仍然严重依赖 TF 1.x summary API 日志操作（如 `tf.compat.v1.summary.scalar()` ）的模型代码用户更容易迁移到TF 2.x，可以首先仅迁移写入API，允许稍后完全迁移模型代码中的单个TF 1.x摘要操作。

为了支持这种迁移风格，`tf.compat.v1.summary` 将在以下条件下自动转发到 TF 2.x：

- 最外层的上下文是 eager 模式
- 设置了默认的TF 2.x summary writer
- 为 wruter 设置了 step 的非空值(使用 `tf.summary.SummaryWriter.as_default`, `tf.summary.experimental.set_step` 或者 `tf.compat.v1.train.create_global_step`)


请注意，当调用TF 2.x摘要实现时，返回值将是一个空的 bytestring 张量，以避免重复的摘要写入。此外，输入参数 forwarding 是尽最大努力，并非所有参数都将被保留（例如，将支持 `family` 参数，而 `collections` 将被删除）。

在 `tf.compat.v1.summary.scalar` 中调用 `tf.summary.scalar` 行为的示例：

In [10]:
tf.compat.v1.enable_v2_behavior()

writer = tf.summary.create_file_writer("/tmp/mylogs/enable_v2_in_v1")
with writer.as_default(step=0):
    tf.compat.v1.summary.scalar('float', tf.constant(1.0), family="family")

In [ ]:
# 